# 6. 累计与分组

深入了解数据是数据挖掘中最重要的工作之一，Pandas提供了一些有效的数据累计，以及结合group by语句进行分组累计的功能，本节我们将进行学习。

我们将通过 Seaborn 程序库中一份行星数据来演示，其中包含天文学家观测到的围绕恒星运转的行星数据。

首先，先通过 Seaborn 下载数据集：

In [ ]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

In [ ]:
planets.head()

## 6.1 描述数据集

DataFrame 提供了非常方便的 describe() 方法来描述数据集的基本情况，在调用之前应该先用 dropna() 方法去掉有缺失值的行：

In [ ]:
planets.dropna().describe()

Pandas 提供的主要累计方法有 count()、first()、last()、mean()、median()、min()、max()、std()、var()、mad()（均值绝对偏差）、prod()（所有项乘积）、sum()（所有项求和）。

## 6.2 分组统计

DataFrame 和 Series 对象都支持 groupby() 方法，它可以分类统计各列的取值。

<b>方法一：按列取值</b>

groupby() 方法返回的是一个 DataFrameGroupBy 对象，在进行下一步聚合运算之前，该对象不会计算。比如，下面代码可以获得不同方法下所有行星公转周期（按天计算）的中位数。

In [ ]:
planets.groupby('method')['orbital_period'].median()

<b>方法二：按组迭代</b>

GroupBy 对象支持直接按组进行迭代，返回的每一组都是 Series 或 DataFrame。下面这个例子统计每种方法有多少条记录。

In [ ]:
for (method, group) in planets.groupby('method'):
    # {0:30s} 用于实现两端对齐
    print("{0:30s} shape={1}".format(method, group.shape))

<b>方法三：调用方法</b>

任意 DataFrame / Series 的方法都可以由 GroupBy 方法调用，从而实现非常灵活强大的操作。

In [ ]:
# 原书案例中最后的 .unstack() 可以省略
planets.groupby('method')['year'].describe()

## 6.3 aggregate()、filter()、transform()、apply()

这四个操作在数据分组之前实现了大量高效的操作。

In [ ]:
import numpy as np
import pandas as pd

rng = np.random.RandomState(0)
df = pd.DataFrame({'key':['A','B','C','A','B','C'], 'data1':range(6), 'data2':rng.randint(0,10,6)},
                 columns=['key','data1','data2'])

<b>1、aggregate()</b>

aggregate() 可以支持复杂的操作，比如字符串、函数或者函数列表，并且能一次性计算所有累计值。

In [ ]:
df.groupby('key').aggregate(['min', np.median, max])

另一种方法是通过字典指定不同列需要累计的函数

In [31]:
df.groupby('key').sum()

,data1,data2
key,,
A,3,8
B,5,7
C,7,12


In [ ]:
df.groupby('key').aggregate({'data1': 'min', 'data2':'max'})

<b>2、filter()</b>

filter() 可以按照分组的属性丢弃若干数据，比如，只保留标准差超过某个阈值的组：

In [ ]:
def filter_func(x):
    return x['data2'].std() > 4

print(df); print(df.groupby('key').std()); 
print(df.groupby('key').filter(filter_func))

<b>3、transform()</b>

transform() 会将数据经过转换，其形状与原来的输入数据是一样的，常见的例子就是将每一组的样本数据减去<b><font color=red>各组</font></b>的平均值，实现数据标准化：

In [ ]:
print(df)
df.groupby('key').transform(lambda x: x - x.mean())

<b>4、apply()</b>

可以在每个组上应用任意方法，该方法输入为分组后的 DataFrame，返回一个 DataFrame、Series或者标量 scalar。

In [32]:
def norm_by_data2(x):
        x['data1'] /= x['data2'].sum()
        return x
    
print(df); print(df.groupby('key').apply(norm_by_data2))

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9
  key     data1  data2
0   A  0.000000      5
1   B  0.142857      0
2   C  0.166667      3
3   A  0.375000      3
4   B  0.571429      7
5   C  0.416667      9


## 6.4 设置分组的键

除了用列名进行分组，还可以用列表、数组、Series、索引作为分组的键。

<b>1、将列表、数组、Series、索引作为分组的键</b>

分组键可以是长度与 DataFrame 匹配的任意 Series 或者列表。

In [33]:
L = [0,1,0,1,2,0]
print(df); print(df.groupby(L).sum())

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9
   data1  data2
0      7     17
1      4      3
2      4      7


<b>2、用字典或Series将索引映射到分组名称</b>

另一种方式是提供一个字典，将索引映射到分组键：

In [35]:
df2 = df.set_index('key')

mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
print(df2); print(df2.groupby(mapping).sum())

     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9
           data1  data2
consonant     12     19
vowel          3      8


<b>3、任意 Python 函数</b>

可以将任意 Python 函数传入 groupby，函数映射到索引，然后新的分组输出：

In [38]:
print(df2); print(df2.groupby(str.lower).mean())

     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9
   data1  data2
a    1.5    4.0
b    2.5    3.5
c    3.5    6.0


<b>4、多个有效键构成的列表</b>

任意有效的键都可以组合起来进行分组，从而返回一个多级索引的分组结果：

In [40]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


## 6.5 分组案例

通过分组，查询不同方法在不同年份发现的行星数量：

In [56]:
decade = 10 * (planets['year']//10)
decade = decade.astype(str)+'s'
decade.name = 'decade'

planets.groupby(['method',decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
